In [1]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout, Add, LSTM, GRU, RNN, Layer
from keras import backend as K
from keras.optimizers import Adam,SGD
import tensorflow as tf
from keras import Model, regularizers, activations, initializers
from keras.constraints import Constraint
import pickle

# load and save mat file
import h5py
import hdf5storage

In [2]:
with h5py.File('test_collected_data.mat', 'r') as file:
    U = file['U'][:]
    U = np.array(U)

    Y = file['Y'][:]
    Y = np.array(Y)

In [3]:
# get X and y data for training and testing

U_input = list()
Y_output = list()

for i in range(U.shape[0]-1):
    if i % 5 == 0:
        U_input.append(U[i:i+5, :])
        Y_output.append(Y[i:i+5, :])

In [4]:
U_input = np.array(U_input)
print(U_input.shape)
Y_output = np.array(Y_output)
print(Y_output.shape)

(4000, 5, 2)
(4000, 5, 2)


In [5]:
RNN_input = np.concatenate((U_input, -U_input), axis=2)
print("RNN_input shape is {}".format(RNN_input.shape))

RNN_input shape is (4000, 5, 4)


In [6]:
RNN_output = Y_output
print("RNN_output shape is {}".format(RNN_output.shape))

RNN_output shape is (4000, 5, 2)


In [7]:
DEFAULT_EPS_SPECTRAL = 1e-3
DEFAULT_MAXITER_SPECTRAL = 10
SWAP_MEMORY = True
STOP_GRAD_SPECTRAL = True

def normalize_kernel(
    kernel,
    u,
    adjustment_coef,
    eps_spectral=DEFAULT_EPS_SPECTRAL,
    maxiter_spectral=DEFAULT_MAXITER_SPECTRAL,
):
    """
    Perform spectral normalization to the kernel given as input. It
    apply the power method to find the largest singular value and apply the spectral
    normalization.

    Args:
        kernel (tf.Tensor): the kernel to orthogonalize
        u (tf.Tensor): the vector used to do the power iteration method
        adjustment_coef (float): the adjustment coefficient as used in convolution
        eps_spectral (float): stopping criterion in spectral algorithm
        maxiter_spectral (int): maximum number of iterations for the power iteration

    Returns:
        tf.Tensor: the normalized kernel, the new u, and sigma which is the largest
            singular value

    """
    W_shape = kernel.shape

    # Flatten the Tensor
    W_reshaped = tf.reshape(kernel, [-1, W_shape[-1]])

    W_bar, u, sigma = spectral_normalization(
        W_reshaped, u, eps=eps_spectral, maxiter=maxiter_spectral
    )

    W_bar = W_bar * adjustment_coef
    W_bar = K.reshape(W_bar, kernel.shape)
    return W_bar, u, sigma

def _power_iteration(
    linear_operator,
    adjoint_operator,
    u,
    eps=DEFAULT_EPS_SPECTRAL,
    maxiter=DEFAULT_MAXITER_SPECTRAL,
    axis=None,
):
    """Internal function that performs the power iteration algorithm to estimate the
    largest singular vector of a linear operator.

    Args:
        linear_operator (Callable): a callable object that maps a linear operation.
        adjoint_operator (Callable): a callable object that maps the adjoint of the
            linear operator.
        u (tf.Tensor): initialization of the singular vector.
        eps (float, optional): stopping criterion of the algorithm, when
            norm(u[t] - u[t-1]) is less than eps. Defaults to DEFAULT_EPS_SPECTRAL.
        maxiter (int, optional): maximum number of iterations for the algorithm.
            Defaults to DEFAULT_MAXITER_SPECTRAL.
        axis (int/list, optional): dimension along which to normalize. Can be set for
            depthwise convolution for example. Defaults to None.

    Returns:
        tf.Tensor: the maximum singular vector.
    """

    # Prepare while loop variables
    u = tf.math.l2_normalize(u, axis=axis)
    # create a fake old_w that doesn't pass the loop condition, it will be overwritten
    old_u = u + 2 * eps

    # Loop body
    def body(u, old_u):
        old_u = u
        v = linear_operator(u)
        u = adjoint_operator(v)

        u = tf.math.l2_normalize(u, axis=axis)

        return u, old_u

    # Loop stopping condition
    def cond(u, old_u):
        return tf.linalg.norm(u - old_u) >= eps

    # Run the while loop
    u, _ = tf.while_loop(
        cond,
        body,
        (u, old_u),
        maximum_iterations=maxiter,
        swap_memory=SWAP_MEMORY,
    )

    # Prevent gradient to back-propagate into the while loop
    if STOP_GRAD_SPECTRAL:
        u = tf.stop_gradient(u)

    return u

def spectral_normalization(
    kernel, u, eps=DEFAULT_EPS_SPECTRAL, maxiter=DEFAULT_MAXITER_SPECTRAL
):
    """
    Normalize the kernel to have its maximum singular value equal to 1.

    Args:
        kernel (tf.Tensor): the kernel to normalize, assuming a 2D kernel.
        u (tf.Tensor): initialization of the maximum singular vector.
        eps (float, optional): stopping criterion of the algorithm, when
            norm(u[t] - u[t-1]) is less than eps. Defaults to DEFAULT_EPS_SPECTRAL.
        maxiter (int, optional): maximum number of iterations for the algorithm.
            Defaults to DEFAULT_MAXITER_SPECTRAL.

    Returns:
        the normalized kernel, the maximum singular vector, and the maximum singular
            value.
    """

    if u is None:
        u = tf.random.uniform(
            shape=(1, kernel.shape[-1]), minval=0.0, maxval=1.0, dtype=kernel.dtype
        )

    def linear_op(u):
        return u @ tf.transpose(kernel)

    def adjoint_op(v):
        return v @ kernel

    u = _power_iteration(linear_op, adjoint_op, u, eps, maxiter)

    # Compute the largest singular value and the normalized kernel.
    # We assume that in the worst case we converged to sigma + eps (as u and v are
    # normalized after each iteration)
    # In order to be sure that operator norm of normalized kernel is strictly less than
    # one we use sigma + eps
    sigma = tf.reshape(tf.norm(linear_op(u)), (1, 1))
    normalized_kernel = kernel / (sigma + eps)
    return normalized_kernel, u, sigma

In [8]:
class SpectralConstraint(Constraint):
    def __init__(
        self,
        k_coef_lip=1.0,
        eps_spectral=DEFAULT_EPS_SPECTRAL,
        u=None,
    ) -> None:
        """
        Ensure that maximum singular values of the weight matrix less than 1.

        Args:
            k_coef_lip (float): lipschitz coefficient of the weight matrix
            eps_spectral (float): stopping criterion for the iterative power algorithm.
            u (tf.Tensor): vector used for iterated power method, can be set to None
                (used for serialization/deserialization purposes).
        """
        self.eps_spectral = eps_spectral
        self.k_coef_lip = k_coef_lip
        if not (isinstance(u, tf.Tensor) or (u is None)):
            u = tf.convert_to_tensor(u)
        self.u = u
        super(SpectralConstraint, self).__init__()

    def __call__(self, w):
        # clip to ensure non-negative weight
        w = K.clip(w, 0, w)

        # perform spectral normalization to the kernel
        wbar, _, _ = normalize_kernel(
            w,
            self.u,
            self.k_coef_lip,
            self.eps_spectral,
        )

        return wbar

    def get_config(self):
        config = {
            "k_coef_lip": self.k_coef_lip,
            "eps_spectral": self.eps_spectral,
            "u": None if self.u is None else self.u.numpy(),
        }
        base_config = super(SpectralConstraint, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [9]:
# set the seed for reproducibility
tf.random.set_seed(42)

# define parameter values
num_step = 5
num_dims = 4

# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(RNN_input, RNN_output, test_size=0.1, random_state=123)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# define scalers for both X and y base on training data only
scaler_X = preprocessing.MinMaxScaler().fit(X_train.reshape(-1, num_dims))
scaler_y = preprocessing.MinMaxScaler().fit(y_train.reshape(-1, 2))

X_train = scaler_X.transform(X_train.reshape(-1, num_dims)).reshape(-1,num_step,num_dims)
X_test = scaler_X.transform(X_test.reshape(-1, num_dims)).reshape(-1,num_step,num_dims)
y_train = scaler_y.transform(y_train.reshape(-1,2)).reshape(-1,num_step,2)
y_test_normalized = scaler_y.transform(y_test.reshape(-1,2)).reshape(-1,num_step,2)

# ICLRNN model training and evaluation
nh = 64
constraint = SpectralConstraint()
input = Input(shape=(X_train.shape[1],X_train.shape[2]))
x = SimpleRNN(nh,
      activation='relu',
      kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.01),
      recurrent_initializer=initializers.Orthogonal(0.1),
      return_sequences=True,
      kernel_constraint=constraint,
      recurrent_constraint=constraint
      )(input)
x = SimpleRNN(nh,
      activation='relu',
      kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.01),
      recurrent_initializer=initializers.Orthogonal(0.1),
      return_sequences=True,
      kernel_constraint=constraint,
      recurrent_constraint=constraint
      )(x)
x = Dense(2,
        activation='relu',
        kernel_constraint=constraint
        )(x)
model = Model(input, x)

model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanSquaredError()])
history = model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_split=0, verbose=2)

loss = model.evaluate(X_test, y_test_normalized, batch_size=256)
test_loss = loss[0]
print(test_loss)

model.summary()

(3600, 5, 4)
(400, 5, 4)
(3600, 5, 2)
(400, 5, 2)
Epoch 1/1000
15/15 - 2s - loss: 6.5775 - mean_squared_error: 6.5775 - 2s/epoch - 124ms/step
Epoch 2/1000
15/15 - 0s - loss: 3.4703 - mean_squared_error: 3.4703 - 63ms/epoch - 4ms/step
Epoch 3/1000
15/15 - 0s - loss: 1.4865 - mean_squared_error: 1.4865 - 97ms/epoch - 6ms/step
Epoch 4/1000
15/15 - 0s - loss: 0.6310 - mean_squared_error: 0.6310 - 57ms/epoch - 4ms/step
Epoch 5/1000
15/15 - 0s - loss: 0.3105 - mean_squared_error: 0.3105 - 58ms/epoch - 4ms/step
Epoch 6/1000
15/15 - 0s - loss: 0.1939 - mean_squared_error: 0.1939 - 74ms/epoch - 5ms/step
Epoch 7/1000
15/15 - 0s - loss: 0.1489 - mean_squared_error: 0.1489 - 73ms/epoch - 5ms/step
Epoch 8/1000
15/15 - 0s - loss: 0.1302 - mean_squared_error: 0.1302 - 125ms/epoch - 8ms/step
Epoch 9/1000
15/15 - 0s - loss: 0.1217 - mean_squared_error: 0.1217 - 193ms/epoch - 13ms/step
Epoch 10/1000
15/15 - 0s - loss: 0.1173 - mean_squared_error: 0.1173 - 136ms/epoch - 9ms/step
Epoch 11/1000
15/15 - 0s 

Epoch 89/1000
15/15 - 0s - loss: 0.0215 - mean_squared_error: 0.0215 - 187ms/epoch - 12ms/step
Epoch 90/1000
15/15 - 0s - loss: 0.0213 - mean_squared_error: 0.0213 - 109ms/epoch - 7ms/step
Epoch 91/1000
15/15 - 0s - loss: 0.0211 - mean_squared_error: 0.0211 - 80ms/epoch - 5ms/step
Epoch 92/1000
15/15 - 0s - loss: 0.0209 - mean_squared_error: 0.0209 - 117ms/epoch - 8ms/step
Epoch 93/1000
15/15 - 0s - loss: 0.0207 - mean_squared_error: 0.0207 - 65ms/epoch - 4ms/step
Epoch 94/1000
15/15 - 0s - loss: 0.0206 - mean_squared_error: 0.0206 - 83ms/epoch - 6ms/step
Epoch 95/1000
15/15 - 0s - loss: 0.0204 - mean_squared_error: 0.0204 - 120ms/epoch - 8ms/step
Epoch 96/1000
15/15 - 0s - loss: 0.0202 - mean_squared_error: 0.0202 - 82ms/epoch - 5ms/step
Epoch 97/1000
15/15 - 0s - loss: 0.0200 - mean_squared_error: 0.0200 - 107ms/epoch - 7ms/step
Epoch 98/1000
15/15 - 0s - loss: 0.0199 - mean_squared_error: 0.0199 - 134ms/epoch - 9ms/step
Epoch 99/1000
15/15 - 0s - loss: 0.0197 - mean_squared_error: 0

Epoch 176/1000
15/15 - 0s - loss: 0.0135 - mean_squared_error: 0.0135 - 83ms/epoch - 6ms/step
Epoch 177/1000
15/15 - 0s - loss: 0.0134 - mean_squared_error: 0.0134 - 82ms/epoch - 5ms/step
Epoch 178/1000
15/15 - 0s - loss: 0.0134 - mean_squared_error: 0.0134 - 83ms/epoch - 6ms/step
Epoch 179/1000
15/15 - 0s - loss: 0.0133 - mean_squared_error: 0.0133 - 149ms/epoch - 10ms/step
Epoch 180/1000
15/15 - 0s - loss: 0.0133 - mean_squared_error: 0.0133 - 117ms/epoch - 8ms/step
Epoch 181/1000
15/15 - 0s - loss: 0.0133 - mean_squared_error: 0.0133 - 61ms/epoch - 4ms/step
Epoch 182/1000
15/15 - 0s - loss: 0.0132 - mean_squared_error: 0.0132 - 68ms/epoch - 5ms/step
Epoch 183/1000
15/15 - 0s - loss: 0.0132 - mean_squared_error: 0.0132 - 83ms/epoch - 6ms/step
Epoch 184/1000
15/15 - 0s - loss: 0.0132 - mean_squared_error: 0.0132 - 97ms/epoch - 6ms/step
Epoch 185/1000
15/15 - 0s - loss: 0.0131 - mean_squared_error: 0.0131 - 70ms/epoch - 5ms/step
Epoch 186/1000
15/15 - 0s - loss: 0.0131 - mean_squared_e

Epoch 263/1000
15/15 - 0s - loss: 0.0112 - mean_squared_error: 0.0112 - 84ms/epoch - 6ms/step
Epoch 264/1000
15/15 - 0s - loss: 0.0112 - mean_squared_error: 0.0112 - 100ms/epoch - 7ms/step
Epoch 265/1000
15/15 - 0s - loss: 0.0111 - mean_squared_error: 0.0111 - 67ms/epoch - 4ms/step
Epoch 266/1000
15/15 - 0s - loss: 0.0111 - mean_squared_error: 0.0111 - 133ms/epoch - 9ms/step
Epoch 267/1000
15/15 - 0s - loss: 0.0111 - mean_squared_error: 0.0111 - 178ms/epoch - 12ms/step
Epoch 268/1000
15/15 - 0s - loss: 0.0110 - mean_squared_error: 0.0110 - 121ms/epoch - 8ms/step
Epoch 269/1000
15/15 - 0s - loss: 0.0110 - mean_squared_error: 0.0110 - 133ms/epoch - 9ms/step
Epoch 270/1000
15/15 - 0s - loss: 0.0110 - mean_squared_error: 0.0110 - 56ms/epoch - 4ms/step
Epoch 271/1000
15/15 - 0s - loss: 0.0110 - mean_squared_error: 0.0110 - 95ms/epoch - 6ms/step
Epoch 272/1000
15/15 - 0s - loss: 0.0109 - mean_squared_error: 0.0109 - 131ms/epoch - 9ms/step
Epoch 273/1000
15/15 - 0s - loss: 0.0109 - mean_squar

Epoch 350/1000
15/15 - 0s - loss: 0.0089 - mean_squared_error: 0.0089 - 100ms/epoch - 7ms/step
Epoch 351/1000
15/15 - 0s - loss: 0.0089 - mean_squared_error: 0.0089 - 194ms/epoch - 13ms/step
Epoch 352/1000
15/15 - 0s - loss: 0.0089 - mean_squared_error: 0.0089 - 230ms/epoch - 15ms/step
Epoch 353/1000
15/15 - 0s - loss: 0.0089 - mean_squared_error: 0.0089 - 78ms/epoch - 5ms/step
Epoch 354/1000
15/15 - 0s - loss: 0.0089 - mean_squared_error: 0.0089 - 67ms/epoch - 4ms/step
Epoch 355/1000
15/15 - 0s - loss: 0.0089 - mean_squared_error: 0.0089 - 82ms/epoch - 5ms/step
Epoch 356/1000
15/15 - 0s - loss: 0.0089 - mean_squared_error: 0.0089 - 99ms/epoch - 7ms/step
Epoch 357/1000
15/15 - 0s - loss: 0.0089 - mean_squared_error: 0.0089 - 98ms/epoch - 7ms/step
Epoch 358/1000
15/15 - 0s - loss: 0.0088 - mean_squared_error: 0.0088 - 133ms/epoch - 9ms/step
Epoch 359/1000
15/15 - 0s - loss: 0.0088 - mean_squared_error: 0.0088 - 150ms/epoch - 10ms/step
Epoch 360/1000
15/15 - 0s - loss: 0.0088 - mean_squa

...

Epoch 611/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 114ms/epoch - 8ms/step
Epoch 612/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 150ms/epoch - 10ms/step
Epoch 613/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 68ms/epoch - 5ms/step
Epoch 614/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 84ms/epoch - 6ms/step
Epoch 615/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 145ms/epoch - 10ms/step
Epoch 616/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 135ms/epoch - 9ms/step
Epoch 617/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 67ms/epoch - 4ms/step
Epoch 618/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 85ms/epoch - 6ms/step
Epoch 619/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 130ms/epoch - 9ms/step
Epoch 620/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 134ms/epoch - 9ms/step
Epoch 621/1000
15/15 - 0s - loss: 0.0083 - mean_squa

Epoch 698/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 59ms/epoch - 4ms/step
Epoch 699/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 67ms/epoch - 4ms/step
Epoch 700/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 82ms/epoch - 5ms/step
Epoch 701/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 162ms/epoch - 11ms/step
Epoch 702/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 88ms/epoch - 6ms/step
Epoch 703/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 100ms/epoch - 7ms/step
Epoch 704/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 147ms/epoch - 10ms/step
Epoch 705/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 87ms/epoch - 6ms/step
Epoch 706/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 83ms/epoch - 6ms/step
Epoch 707/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 68ms/epoch - 5ms/step
Epoch 708/1000
15/15 - 0s - loss: 0.0083 - mean_squared

Epoch 785/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 132ms/epoch - 9ms/step
Epoch 786/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 117ms/epoch - 8ms/step
Epoch 787/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 60ms/epoch - 4ms/step
Epoch 788/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 58ms/epoch - 4ms/step
Epoch 789/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 82ms/epoch - 5ms/step
Epoch 790/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 100ms/epoch - 7ms/step
Epoch 791/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 62ms/epoch - 4ms/step
Epoch 792/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 70ms/epoch - 5ms/step
Epoch 793/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 131ms/epoch - 9ms/step
Epoch 794/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 88ms/epoch - 6ms/step
Epoch 795/1000
15/15 - 0s - loss: 0.0083 - mean_squared_

Epoch 872/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 145ms/epoch - 10ms/step
Epoch 873/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 166ms/epoch - 11ms/step
Epoch 874/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 104ms/epoch - 7ms/step
Epoch 875/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 70ms/epoch - 5ms/step
Epoch 876/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 59ms/epoch - 4ms/step
Epoch 877/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 87ms/epoch - 6ms/step
Epoch 878/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 101ms/epoch - 7ms/step
Epoch 879/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 100ms/epoch - 7ms/step
Epoch 880/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 147ms/epoch - 10ms/step
Epoch 881/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 135ms/epoch - 9ms/step
Epoch 882/1000
15/15 - 0s - loss: 0.0083 - mean_sq

Epoch 959/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 68ms/epoch - 5ms/step
Epoch 960/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 66ms/epoch - 4ms/step
Epoch 961/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 104ms/epoch - 7ms/step
Epoch 962/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 91ms/epoch - 6ms/step
Epoch 963/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 99ms/epoch - 7ms/step
Epoch 964/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 194ms/epoch - 13ms/step
Epoch 965/1000
15/15 - 0s - loss: 0.0084 - mean_squared_error: 0.0084 - 154ms/epoch - 10ms/step
Epoch 966/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 102ms/epoch - 7ms/step
Epoch 967/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 133ms/epoch - 9ms/step
Epoch 968/1000
15/15 - 0s - loss: 0.0083 - mean_squared_error: 0.0083 - 74ms/epoch - 5ms/step
Epoch 969/1000
15/15 - 0s - loss: 0.0083 - mean_squar

In [10]:
# save model and scalers
model.save('iclrnn_energy.keras')
pickle.dump(scaler_X, open('iclrnn_scaler_X', 'wb'))
pickle.dump(scaler_y, open('iclrnn_scaler_y', 'wb'))